In [2]:
using IntervalArithmetic
using LinearAlgebra
using Plots
# <img src="img/force_graph_annotated_copy.png" alt="Force graph annotated" style="width: 600px;"/>

# Code for computing exactly the solution to the piecewise linear ODE using interval arithmetic

There are six piecewise linear pieces

![my_image](img/force_graph_annotated_copy.png)

In reality, we take $h^2 = 1.5$, $c = 0.05$, $\ell = 5$

Note that the piece $P_0$ has slope 1. 

For these parameters, starting at the end of the flat piece, that is, $y = 9$, passes over $y = 0$

But starting beyond that, in particular, $y = 15$ does not. 

We write code to find 
- $T_0, v_0$ in terms of $b$. For $b=9$, $T_0 = 0$ and $v_0 = 0$. For $b=15$, these values are different.
- $T_1, v_1$ in terms of $T_0, v_0$
- $T_2, v_2$ in terms of $T_1, v_1$
- $T_3, v_3$ in terms of $T_2, v_2$
- $T_4, v_4$ in terms of $T_3, v_3$
- Whether or not $y = 0$ is reached in terms of $T_4, v_4$

All numerics are done using the julia package IntervalArithmetic, a validated numerics package.

The various pieces have the following equations:

- 
$P_0$ has equation 
$$\ddot{Y} + \frac{2}{t}\dot{Y} + Y = 0$$ 
where $Y = y - (4+\ell)$. 
This has initial condition 
$$Y(0) = b,\quad \dot{Y}(0) = 0$$ 
and solution
$$Y(t) = \frac{b\sin(t)}{t}$$
$$\dot{Y}(t) = \frac{b\cos(t)}{t} - \frac{b\sin(t)}{t^2}$$

- $P_1$ has equation 
$$\ddot{Y} + \frac{2}{t}\dot{Y} + c = 0$$
where $Y = y - (4 + \ell)$. 
This has initial condition 
$$Y(T_0) = 0,\quad \dot{Y}(T_0) = v_0$$
and solution
$$Y(t) = -\frac{ct^2}{6} + \frac{k_1}{t} + k_2$$
$$\dot{Y}(t) = -\frac{ct}{3} - \frac{k_1}{t^2}$$

- 
$P_2$ has equation
$$\ddot{Y} + \frac{2}{t}\dot{Y} - (h^2 -c) Y + c = 0$$ 
where $Y = y - 4$.
This has initial condition 
$$Y(T_1) = 0,\quad \dot{Y}(T_1) = v_1$$
and solution
$$Y(t) = \frac{k_1e^{-t\sqrt{h^2-c}}}{t} + \frac{k_2e^{t\sqrt{h^2-c}}}{t} + \frac{c}{h^2-c}$$
$$\dot{Y}(t) = -\frac{k_1\sqrt{h^2-c}e^{-t\sqrt{h^2-c}}}{t} - \frac{k_1e^{-t\sqrt{h^2-c}}}{t^2} + \frac{k_2\sqrt{h^2-c}e^{t\sqrt{h^2-c}}}{t} - \frac{k_2e^{t\sqrt{h^2-c}}}{t^2}$$

- 
$P_3$ has equation
$$\ddot{Y} + \frac{2}{t}\dot{Y} + h^2 (Y+1) = 0 $$ 
where $Y = y - 3$. 
This has initial condition 
$$Y(T_2) = 0,\quad \dot{Y}(T_2) = v_2$$
and solution
$$Y(t) = \frac{k_1 \sin(ht)}{t} + \frac{k_2\cos(ht)}{t}-1$$
$$\dot{Y}(t) = \frac{hk_1\cos(ht)}{t} - \frac{k_1\sin(ht)}{t^2} - \frac{hk_2\sin(ht)}{t} - \frac{k_2\cos(ht)}{t^2}$$

- 
$P_4$ has equation 
$$\ddot{Y} + \frac{2}{t}\dot{Y} + Y = 0$$ 
where $Y = y - 2$. 
This has initial condition 
$$Y(T_3) = 0,\quad \dot{Y}(T_3) = v_3$$
and solution 
$$Y(t) = \frac{k_1\sin(t)}{t} + \frac{k_2\cos(t)}{t}$$
$$\dot{Y}(t) = \frac{k_1\cos(t)}{t} - \frac{k_1\sin(t)}{t^2} - \frac{k_2\sin(t)}{t} - \frac{k_2\cos(t)}{t^2}$$

- 
$P_5$ has equation 
$$\ddot{y} + \frac{2}{t}\dot{Y} - Y - 1 = 0$$ 
where $Y = y - 1$. 
This has initial condition 
$$Y(T_4) = 0,\quad \dot{Y}(T_4) = v_4$$
and solution 
$$Y(t) = \frac{k_1e^{t}}{t} + \frac{k_2e^{-t}}{t} - 1$$
$$\dot{Y}(t) = \frac{k_1e^t}{t} - \frac{k_1e^t}{t^2} - \frac{k_2e^{-t}}{t} - \frac{k_2e^{-t}}{t^2}$$

We pass over $y = 0$ if $k_1 < 0$, and stay in the region $y > 0$ if $k_1 > 0$. 

In [3]:
h2 = 1.5
h = sqrt(h2)
c = 0.05
l = 5

5

In [4]:
M = [1 2 3; 4 5 6]

2×3 Array{Int64,2}:
 1  2  3
 4  5  6

In [5]:
nullspace(M)

3×1 Array{Float64,2}:
  0.40824829046386285
 -0.816496580927726  
  0.40824829046386313

In [6]:
a = @interval(1.)
a isa Interval

true

In [74]:
# takes in a matrix [A11 A12 A13; A21 A22 A23]
# solves for k_1 A11 + k_2 A12 + A13 = 0 and
# k_1 A21 + k_2 A22 + A23 = 0

function solve_affine_system(M)
#     N = nullspace(M)
#     N ./= N[3]
#     return N[1], N[2]
    k2 = (M[1,1] * M[2,3] - M[2,1] * M[1,3]) / (M[2,1] * M[1,2] - M[1,1] * M[2,2])
    k1 = (- M[1,3] - M[1,2] * k2) / M[1,1]
    
    return k1,k2
end

MAX_VAL = 1000
# use a binary search
# we assume initial position is GREATER than the target
function solve_for_time(odefunc, target, initial_time, inc = 0.1, tol = 1e-6)
#     print(initial_time)
    y0 = odefunc(initial_time)
    Tmax = initial_time
    # increment time until we first pass
    while odefunc(Tmax) > target
        Tmax += 0.1
#         println(odefunc(Tmax))
#         flush(stdout)
        if odefunc(Tmax) > y0
            return -1
        end
    end
    
    l = initial_time
    u = Tmax
    m = (l+u)/2
    # at all times, func(u) < target, func(l) > target
    while u - l > tol
        m = (u+l) / 2
#         println(l, ", ", u)
        mv = odefunc(m)

            
        if mv < target
            u = m
        else
            l = m
        end
    end
    if target isa Interval
        return @interval(l, u)
    else
        return m
    end
end

solve_for_time (generic function with 3 methods)

In [8]:
# y(t) = bsin(t)/t
# must solve bsin(t)/t = 0, this occurs for the first time at t = pi/2
# NOTE: this is wrong updated version below agrees with python code & is accurate
function find_t0v0(b)
    d = b - (4+l)
    T0 = pi
    v0 = -d/T0
    return T0, v0
end

# must solve solve for:
# -cT_0^2/6 + k_1/t + k_2 = 0
# -cT_0/3 - k_1/t^2 = v0
# first solve for k_1, then k_2
function find_t1v1(T0, v0)
    # first solve for k1 and k2 so we know Y(t)
    M = [1/T0 1 -c * T0^2/6 ; -1/(T0^2) 0 (-c * T0 / 3 - v0)]
    k1, k2 = solve_affine_system(M)

    f(t)  = -(c * t^2) / 6 + k1 / t + k2
    fd(t) = -c * t / 3 - k1 / (t^2)
    
    println("T0, v0: ", T0, ", ", v0)
    println("f(T0), fd(T0): ", f(T0), ", ", fd(T0))
#     return f, fd
#     then solve for T1 such that Y(T1) = -4 - l using a binary search, as Y decreases monotonically
    T1 = solve_for_time(f, -l, T0)
    
    
    # then solve for v1 at T1 using the formula for Y'(T1)
    v1 = fd(T1)
    
    return T1, v1
end

function find_t1v1_for_start_topramp()
    k1 = 0
    k2 = 0
    f(t)  = -(c * t^2) / 6 + k1 / t + k2
    fd(t) = -c * t / 3 - k1 / (t^2)
    T1 = solve_for_time(f, -l, T0)
    v1 = fd(T1)
    return T1, v1
end

# equations are complicated so I will not write them here
function find_t2v2(T1, v1)
    k1_coeff_1 = exp(-T1 * sqrt(h2 - c))/T1 
    k2_coeff_1 = exp(T1 * sqrt(h2 - c))/T1
    const1 = c/(h2-c)
    
    sd = sqrt(h2 - c)
    
    k1_coeff_2 = -sqrt(h2-c) * exp(-T1 * sd) / T1
    k1_coeff_2 -= exp(-T1 * sd)/(T1^2)
    
    k2_coeff_2 = sqrt(h2-c) * exp(T1 * sd) / T1
    k2_coeff_2 -= exp(T1 * sd) / (T1^2)
    const2 = -v1
    
    M = [k1_coeff_1 k2_coeff_1 const1; k1_coeff_2 k2_coeff_2 const2]
    k1, k2 = solve_affine_system(M)
    
    f(t) = k1 * exp(-t * sd) / t + k2 * exp(t * sd) / t + c / (h2 - c)
    fd(t) = - k1 * sd * exp(-t * sd) / t - k1 * exp(-t * sd) / (t^2) + k2 * sd * exp(t * sd) / t - k2 * exp(t*sd)/(t^2)
    
    
    println("T1, v1: ", T1, ", ", v1)
    println("f(T1), fd(T1): ", f(T1), ", ", fd(T1))
    
    T2 = solve_for_time(f, -1, T1)
    v2 = fd(T2)
    return T2, v2
end

# Y(t) = k1 sin(t) / t + k2 cos(t) / t - 1
function find_t3v3(T2, v2)
    k1_coeff_1 = sin(h * T2)/T2
    k2_coeff_1 = cos(h * T2)/T2
    const1 = -1
    
    k1_coeff_2 = h * cos(h * T2)/T2 - sin(h * T2)/(T2^2)
    k2_coeff_2 = -h * sin(h * T2)/T2 - cos(h * T2)/(T2^2)
    const2 = -v2
/    
    M = [k1_coeff_1 k2_coeff_1 const1; k1_coeff_2 k2_coeff_2 const2]
    k1, k2 = solve_affine_system(M)
    
    f(t) = k1 * sin(h * t) / t + k2 * cos(h * t) / t - 1
    fd(t) = k1 * h * cos(h * t) / t - k1 * sin(h * t) / (t^2) - 
            k2 * h * sin(h * t) / t - k2 * cos(h * t) / t^2
    
    println(T2, ", ", v2)
    println(f(T2), ", ", fd(T2))
    
    T3 = solve_for_time(f, -1, T2)
    v3 = fd(T3)
    
    return T3, v3
    
end

function find_t4v4(T3, v3)
    k1_coeff_1 = sin(T3)/T3
    k2_coeff_1 = cos(T3)/T3
    const1 = 0
    
    k1_coeff_2 = cos(T3)/T3 - sin(T3)/(T3^2)
    k2_coeff_2 = -sin(T3)/T3 - cos(T3)/(T3^2)
    const2 = -v3
/    
    M = [k1_coeff_1 k2_coeff_1 const1; k1_coeff_2 k2_coeff_2 const2]
    k1, k2 = solve_affine_system(M)
    
    f(t) = k1 * sin(t) / t + k2 * cos(t) / t
    fd(t) = k1 * cos(t) / t - k1 * sin(t) / (t^2) - 
            k2 * sin(t) / t - k2 * cos(t) / t^2
    
    println("T3, v3: ", T3, ", ", v3)
    println("f(T3), fd(T3): ", f(T3), ", ", fd(T3))
    
    T4 = solve_for_time(f, -1, T3)
    v4 = fd(T4)
    
    return T4, v4
end

function does_it_pass_over(T4,v4)
    k1_coeff_1 = exp(T4)/T4 
    k2_coeff_1 = exp(-T4)/T4
    const1 = -1
    
    k1_coeff_2 = exp(T4)/T4 - exp(T4)/(T4^2) 
    k2_coeff_2 = - exp(-T4)/T4 - exp(-T4)/(T4^2)
    const2 = -v4
    
    M = [k1_coeff_1 k2_coeff_1 const1; k1_coeff_2 k2_coeff_2 const2]
    k1, k2 = solve_affine_system(M)
    
    # if k1 < 0, we pass over; otherwise, we don't
    return k1 < 0
end

function eval_does_it_pass_over(b)
    T0, v0 = find_t0v0(b)
    T1, v1 = find_t1v1(T0, v0)
    T2, v2 = find_t2v2(T1, v1)
    T3, v3 = find_t3v3(T2, v2)
    T4, v4 = find_t4v4(T3, v3)
    return does_it_pass_over(T4, v4)
end

eval_does_it_pass_over (generic function with 1 method)

In [9]:
"""
more general solver for linear klein gordon equation 
y'' + (2/t) y' + my + q = 0 
y(t0) = y0, y'(t0) = v0

RETURNS f(t), v(t), which are y and y' as a function of t. 

if m != 0, substituting Y = y + q/m, we obtain
Y'' + (2/t) Y' + mY = 0

distinguishes between three cases: m < 0, m = 0, m > 0
"""

"""
IF m = 0
y(t) = -qt^2/6 + k1/t + k2 = 0
y'(t) = -qt/3 - k1/t^2
"""
function solve_ode_linear_mzero(t0, y0, v0, q)
    if isapprox(t0, 0)
        @assert(isapprox(v0, 0))
        k1 = 0
        k2 = y0
    else
        k1_coeff1 = 1/t0
        k2_coeff1 = 1
        const1 = -q * t0^2 / 6 - y0
        
        k1_coeff2 = -1/(t0^2)
        k2_coeff2 = 0
        const2 = -q*t0 / 3 - v0
        
        M = [k1_coeff1 k2_coeff1 const1; k1_coeff2 k2_coeff2 const2]
        k1, k2 = solve_affine_system(M)
    end
        
    f(t) = - q * t^2 / 6 + k1 / t + k2
    v(t) = - q * t / 3 - k1 / (t^2)
    
    if isapprox(t0, 0)
        fy0 = k2
        fv0 = 0
    else
        fy0 = f(t0)
        fv0 = v(t0)
    end
        
    
#     @assert(isapprox(fy0, y0))
#     @assert(isapprox(fv0, v0))
    
    return f, v
end

"""
IF m > 0, let m = r^2
y(t) = k1 * sin(rt)/t + k2 * cos(rt) / t - q/m
y'(t) = k1 * r * cos(rt)/t - k1 * sin(rt) / t^2 - k2 * r * sin(rt) / t - k2 * cos(rt) / t^2
"""
function solve_ode_linear_mpos(t0, y0, v0, m, q)
    @assert(m > 0)
    r = sqrt(m)
    
    if isapprox(t0, 0)
        @assert(isapprox(v0, 0))
        # r * k1 - q / m = y0
        k2 = 0
        k1 = q / (m*r) + y0 / r
    else
        k1_coeff1 = sin(r * t0) / t0
        k2_coeff1 = cos(r * t0) / t0
        const1 = -q/m - y0
        
        k1_coeff2 = r * cos(r * t0) / t0 - sin(r * t0) / (t0^2)
        k2_coeff2 = -r * sin(r * t0) / t0 - cos(r * t0) / (t0^2)
        const2 = -v0
        
        M = [k1_coeff1 k2_coeff1 const1 ; k1_coeff2 k2_coeff2 const2]
        k1, k2 = solve_affine_system(M)
    end
    
    f(t) = k1 * sin(r * t) / t + k2 * cos(r * t) / t- q / m
    v(t) = k1 * r * cos(r * t) / t - k1 * sin(r * t) / (t^2) - k2 * r * sin(r * t) / t - k2 * cos(r * t) / (t^2)
    
    if isapprox(t0, 0)
        fy0 = r * k1 - q / m
        fv0 = 0
    else
        fy0 = f(t0)
        fv0 = v(t0)
    end
    
#     @assert(isapprox(fy0, y0))
#     @assert(isapprox(fv0, v0))
    
    return f, v
end

"""
IF m < 0
y(t) = k1 * e^t / t + k2 * e^(-t) / t - q/m
y'(t) = k1 * e^t / t - k1 * e^t / t^2 - k2 * e^(-t) / t - k2 * e^(-t)/t^2
"""
function solve_ode_linear_mneg(t0, y0, v0, m, q)
    @assert(m < 0)
    r = sqrt(-m)
    
    if isapprox(t0, 0)
        @assert(isapprox(v0, 0))
        # k1 = -k2
        # k1 - k2  - q/m = y0
        a = (y0 + (q/m)) / 2
        k1 = a
        k2 = -a
        k1 = q / (m*r) + y0 / r
    else
        k1_coeff1 = exp(r * t0) / t0
        k2_coeff1 = exp(-r * t0) / t0
        const1 = -q/m - y0
        
        k1_coeff2 = r * exp(r * t0) / t0 - exp(r * t0) / (t0^2)
        k2_coeff2 = -r * exp(-r * t0) / t0 - exp(-r * t0) / (t0^2)
        const2 = -v0
        
        M = [k1_coeff1 k2_coeff1 const1 ; k1_coeff2 k2_coeff2 const2]
        k1, k2 = solve_affine_system(M)
    end
    
    f(t) = k1 * exp(r * t) / t + k2 * exp(-r * t) / t - q / m
    v(t) = k1 * r * exp(r * t) / t - k1 * exp(r * t) / (t^2) - k2 * r * exp(-r * t) / t - k2 * exp(-r * t) / (t^2)
    
    if isapprox(t0, 0)
        fy0 = k1 - k2 - q/m
        fv0 = 0
    else
        fy0 = f(t0)
        fv0 = v(t0)
    end
    
#     println(t0, " ", y0, " ", v0, " ", m, " ", q)
#     println(fy0)
#     @assert(isapprox(fy0, y0))
#     @assert(isapprox(fv0, v0))
    
    return f, v
end

function solve_ode_linear(t0, y0, v0, m, q)
    if isapprox(m, 0)
        return solve_ode_linear_mzero(t0, y0, v0, q)
    elseif m > 0
        return solve_ode_linear_mpos(t0, y0, v0, m, q)
    else
        return solve_ode_linear_mneg(t0, y0, v0, m, q)
    end
end



# control points are:
# T0, v0 at 4+l
# T1, v1 at 4
# T2, v2 at 3
# T3, v3 at 2
# T4, v4 at 1
function eval_does_it_pass_over2(b)
    println("STARTING AT: ", b)
    # f(y) = y - (4+l) + c
    f0, v0 = solve_ode_linear(@interval(0), @interval(b), @interval(0), @interval(1), @interval(-4-l+c))
    T0 = solve_for_time(f0, @interval(4+l), @interval(1e-5))
    v0 = v0(T0)
    println("T0, v0: ", T0, ", ", v0)
    
    # f(y) = c
    f1, v1 = solve_ode_linear(T0, 4+l, v0, 0, c)
    T1 = solve_for_time(f1, 4, T0)
    v1 = v1(T1)
    println("T1, v1: ", T1, ", ", v1)
    
    # f(y) = h^2 + (c - h^2) (y - 3)
    # m = (c-h^2)
    # q = h^2 - 3 (c - h^2)
    f2, v2 = solve_ode_linear(T1, 4, v1, c-h2, h2 - 3 * (c - h2))
    T2 = solve_for_time(f2, 3, T1)
    v2 = v2(T2)
    println("T2, v2: ", T2, ", ", v2)
    
    # f(y) = h^2(y-2)
    f3, v3 = solve_ode_linear(T2, 3, v2, h2, -2 * h2)
    T3 = solve_for_time(f3, 2, T2)
    v3 = v3(T3)
    println("T3, v3: ", T3, ", ", v3)
    
    # f(y) = y-2
    f4, v4 = solve_ode_linear(T3, 2, v3, 1, -2)
    T4 = solve_for_time(f4, 1, T3)
    v4 = v4(T4)
    println("T4, v4: ", T4, ", ", v4)
    
    # f(y) = -y
    f5, v5 = solve_ode_linear(T4, 1, v4, -1, 0)
    
    long_time = f5(200)
    return long_time < 0
end

eval_does_it_pass_over2 (generic function with 1 method)

In [78]:
# R0 = line going to infinity
# R1 = plateau
# R2 = uphill
# R3 = downhill to 0
# R4 = downhill to negative
MAXLEN = 10000
# (xmin, xmax), (m, q) where f(y) = my + q
R0 = [(4+l, MAXLEN), (1, -4-l+c)]
R1 = [(4, 4+l), (0, c)]
R2 = [(3, 4), (c-h2, h2 - 3 * (c - h2))]
R3 = [(2, 3), (h2, -2 * h2)]
R4 = [(1, 2), (1, -2)]
R5 = [(0, 1), (-1, 0)]
the_regs = [R0, R1, R2, R3, R4, R5]


function eval_does_it_pass_over_reg(b, v = 0, t = 0, regi=-1)
    if regi == -1
        for i in 1:6
            r = the_regs[i]
            if b > r[1][1] && b <= r[1][2]
                regi = i
                break
            else
                continue
            end
        end
    end
    
    
    r = the_regs[regi]
    
    m = r[2][1]
    q = r[2][2]
#     println(regi, ", ", t, ", ", b, ", ", v)
    f_func, v_func = solve_ode_linear(t, b, v, m, q)
    
    if regi == 6
        return f_func(200) < 0
    else
        next = r[1][1]
        T_new = solve_for_time(f_func, next, max(t, 1e-5))
        if T_new < 0
            return false
        end
        v_new = v_func(T_new)
        return eval_does_it_pass_over_reg(next, v_new, T_new, regi+1)
    end
end

eval_does_it_pass_over_reg (generic function with 4 methods)

In [31]:
f, v = solve_ode_linear_mzero(0, 9, 0, 0.05)

(f, v)

In [32]:
eval_does_it_pass_over(9.1)

T0, v0: π = 3.1415926535897..., -0.03183098861837896
f(T0), fd(T0): 0.0, -0.03183098861837895
T1, v1: 24.832029746966995, -0.41353858273264255
f(T1), fd(T1): 1.3877787807814457e-17, -0.4135385827326426
26.30898921390122, -1.2658866572736351
0.0, -1.2658866572736351
T3, v3: 26.949220823245707, -1.6930659839546913
f(T3), fd(T3): -5.551115123125783e-17, -1.693065983954691


true

In [511]:
eval_does_it_pass_over(15)

T0, v0: π = 3.1415926535897..., -1.909859317102744
f(T0), fd(T0): 0.0, -1.909859317102744
T1, v1: 10.717737015085742, -0.3382250390636018
f(T1), fd(T1): -1.3877787807814457e-17, -0.3382250390636018
12.405571993189374, -1.1962427106571027
0.0, -1.1962427106571025
T3, v3: 13.084885938959637, -1.570686189708663
f(T3), fd(T3): 0.0, -1.5706861897086626


false

In [524]:
eval_does_it_pass_over2(@interval(9.1))

STARTING AT: [9.09999, 9.10001]
T0, v0: [2.27886, 2.27887], [-0.0647494, -0.0647493]
T1, v1: [24.4653, 24.4654], [-0.407989, -0.407988]
T2, v2: [25.9529, 25.953], [-1.26366, -1.26365]
T3, v3: [26.5941, 26.5942], [-1.69056, -1.69054]
T4, v4: [27.2451, 27.2452], [-1.27594, -1.27592]


true

In [516]:
eval_does_it_pass_over2(@interval(15))

STARTING AT: [15, 15]
T0, v0: [3.11583, 3.11584], [-1.9571, -1.95709]
T1, v1: [10.4636, 10.4637], [-0.343329, -0.343328]
T2, v2: [12.1427, 12.1428], [-1.19531, -1.19529]
T3, v3: [12.823, 12.8231], [-1.5671, -1.56708]
T4, v4: [13.5639, 13.564], [-1.01935, -1.01932]


false

In [84]:
L = LinRange(0.01, 30.0, 100)

100-element LinRange{Float64}:
 0.01,0.312929,0.615859,0.918788,…,28.7883,29.0912,29.3941,29.6971,30.0

In [85]:
for b in L
    res = eval_does_it_pass_over_reg(b)
    println(b, ",", res)
    flush(stdout)
end

0.01,false
0.31292929292929295,false
0.6158585858585859,false
0.9187878787878788,false
1.2217171717171718,false
1.5246464646464646,false
1.8275757575757576,false
2.1305050505050502,false
2.4334343434343433,false
2.7363636363636368,false
3.0392929292929294,false
3.342222222222222,false
3.6451515151515155,false
3.9480808080808085,false
4.251010101010101,false
4.553939393939395,false
4.856868686868687,false
5.15979797979798,false
5.462727272727273,true
5.765656565656565,true
6.068585858585859,true
6.371515151515152,true
6.674444444444444,true
6.977373737373737,true
7.280303030303031,true
7.583232323232323,true
7.886161616161617,true
8.18909090909091,true
8.492020202020202,true
8.794949494949494,true
9.09787878787879,true
9.400808080808082,true
9.703737373737374,true
10.006666666666666,true
10.30959595959596,true
10.612525252525252,true
10.915454545454546,true
11.21838383838384,true
11.52131313131313,true
11.824242424242424,true
12.127171717171716,true
12.43010101010101,true
12.73303030303